In [ ]:

#Author : Navneet Kumar
#Co-Author: Ayush jain

In [1]:
import cv2
import mediapipe as mp
import time
import numpy as np

In [2]:
import math
def closed_fingers(lmlist,img):
    fingers = []
    if len(lmlist) > 20:  # Ensure all landmarks are present
        cv2.putText(img, "Hand Detected", (10,50), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,255), 3)
        # Thumb
        if lmlist[4][2] > lmlist[3][2]:
            fingers.append("Thumb")
        # Index finger
        if lmlist[8][2] > lmlist[6][2]:
            fingers.append("Index")
        # Middle finger
        if lmlist[12][2] > lmlist[10][2]:
            fingers.append("Middle")
        # Ring finger
        if lmlist[16][2] > lmlist[14][2]:
            fingers.append("Ring")
        # Pinky
        if lmlist[20][2] > lmlist[18][2]:
            fingers.append("Pinky")

    return fingers

def finger_close_dist(tip,adj,zero):
    #two adjacent points on same finger
    # distance from zero to tip 
    dist1 = math.sqrt((zero[0] - tip[0])**2 + (zero[1] - tip[1])**2)
    dist2 =  math.sqrt((zero[0] - adj[0])**2 + (zero[1] - adj[1])**2)
    if(dist2>=dist1):
        return True
    else:
        return False

def closed_fingers_v2(lmlist,img):
    fingers = []
    if len(lmlist) > 20:  # Ensure all landmarks are present
        cv2.putText(img, "Hand Detected", (10,50), cv2.FONT_HERSHEY_PLAIN, 3, (255,255,255), 3)
        zero= lmlist[0][1],lmlist[0][2]
        # Thumb
        if finger_close_dist(lmlist[4][1:3],lmlist[3][1:3],zero):
            #add extra checks for thumb

            fingers.append("Thumb")
        # Index finger
        if finger_close_dist(lmlist[8][1:3],lmlist[6][1:3],zero):
            fingers.append("Index")
        # Middle finger
        if finger_close_dist(lmlist[12][1:3],lmlist[10][1:3],zero):
            fingers.append("Middle")
        # Ring finger
        if finger_close_dist(lmlist[16][1:3],lmlist[14][1:3],zero):
            fingers.append("Ring")
        # Pinky
        if finger_close_dist(lmlist[20][1:3],lmlist[18][1:3],zero):
            fingers.append("Pinky")
            
    return fingers

def up_fingers(closed_list):
    open_fingers=['Thumb','Index','Middle','Ring','Pinky']
    for i in closed_list:
        open_fingers.remove(i)
    return open_fingers

def ispointing(closedfingers,img):
    if 'Middle' in closedfingers and 'Ring' in closedfingers and 'Pinky' in closedfingers and 'Index' not in closedfingers:
        cv2.putText(img, "Pointing", (10,100), cv2.FONT_HERSHEY_PLAIN, 3, (255,255,255), 3)
        return True
    return False


    

        

In [3]:

class writer:
    def __init__(self):
        self.points=[]
        self.Zdist=-0.05
        self.thres=0.03
        self.dir='bi'
    def calibrate(self,currentZ,thres,dir):
        self.Zdist=currentZ
        self.thres=thres
        self.dir=dir
    def on_canvas(self,point):
        if(self.dir=='bi'):
            if(abs(point.z-self.Zdist)<self.thres):
                return True
            return False
        elif(self.dir=='uni1'):
            if(point.z<self.Zdist):
                return True
            return False
        elif(self.dir=='uni2'):
            if(point.z>self.Zdist):
                return True
            return False
      
    def write(self,point):
        if(self.on_canvas(point)):
            self.points.append(point)
        # if len(self.points)>1:
            # cv2.line(self.img,self.points[-2],self.points[-1],(255,0,0),5)

    def clear(self):
        self.points.clear()
    # def make_line(self,img):
    #     if len(self.points)>1:
    #         for i in range(len(self.points)-1):
    #             cv2.line(self.img,self.points[i],self.points[i+1],(255,0,0),5)
    def draw(self,img):
        if len(self.points)>1:
            for i in range(len(self.points)-1):
                cv2.line(img,(int(self.points[i].x),int(self.points[i].y)),(int(self.points[i+1].x),int(self.points[i+1].y)),(255,0,0),5)
        return img

In [4]:
class Mypoint:
    def __init__(self,x,y,z):
        self.x=x
        self.y=y
        self.z=z


In [5]:
def Draw(img,points):
    if len(points)>1:
        for i in range(len(points)-1):
            cv2.line(img,(int(points[i].x),int(points[i].y)),(int(points[i+1].x),int(points[i+1].y)),(255,0,0),5)
            

In [6]:
cap = cv2.VideoCapture(0)
# flip the image

mpHands = mp.solutions.hands
hands = mpHands.Hands(static_image_mode=False,
                      max_num_hands=1,
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5)
mpDraw = mp.solutions.drawing_utils

pTime = 0
cTime = 0
wr = writer()
wr.calibrate(currentZ=-0.09,thres=0.005,dir='uni1')
Writers=[]
Writers.append(wr)
## POINTER TRACK TABLE
pointer_table=np.zeros((10,2))
last_time_pointer=False
this_time_pointer=True
while True:
 

    try:
        success, img = cap.read()
        img=cv2.flip(img,1)
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
       
  
        results = hands.process(imgRGB)
        #print(results.multi_hand_landmarks)
        lmlist = []
        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:
                for id, lm in enumerate(handLms.landmark):
                    #print(id,lm)
                    h, w, c = img.shape
                    cx, cy = int(lm.x *w), int(lm.y*h)
                    cz= lm.z
                    #if id ==0:
                    # cv2.circle(img, (cx,cy), 3, (255,0,255), cv2.FILLED)
                    # circle with numbers
                    cv2.putText(img, str(id), (cx,cy), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0), 1)
                    lmlist.append([id, cx, cy, cz])

                mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)

            
                

        def closed_fingers(lmlist):
            fingers = []
            if len(lmlist) > 20:  # Ensure all landmarks are present
                cv2.putText(img, "Hand Detected", (10,50), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,255), 3)
                # Thumb
                if lmlist[4][2] > lmlist[3][2]:
                    fingers.append("Thumb")
                # Index finger
                if lmlist[8][2] > lmlist[6][2]:
                    fingers.append("Index")
                # Middle finger
                if lmlist[12][2] > lmlist[10][2]:
                    fingers.append("Middle")
                # Ring finger
                if lmlist[16][2] > lmlist[14][2]:
                    fingers.append("Ring")
                # Pinky
                if lmlist[20][2] > lmlist[18][2]:
                    fingers.append("Pinky")

            return fingers
         

        c_fingers= closed_fingers_v2(lmlist,img)

        Pencil=True



        if(len(c_fingers)>0):
            cv2.putText(img, str(up_fingers(c_fingers)), (10,100), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,200), 1)

            new_point=Mypoint(lmlist[8][1],lmlist[8][2],lmlist[8][3])
            if(Writers[-1].on_canvas(new_point)):
                # glow the point
                cv2.circle(img, (int(new_point.x),int(new_point.y)), 5, (255,150,200), cv2.FILLED)
            # print("pointer ",last_time_pointer," ",this_time_pointer)
            if ispointing(c_fingers,img):
                this_time_pointer=True
                # make new writer when last time not pointing and this time pointing
                if not last_time_pointer and this_time_pointer:
                    new_writer=writer()
                    new_writer.calibrate(currentZ=-0.07,thres=0.005,dir='uni1')
                    Writers.append(new_writer)
                    
                    

                
                cv2.putText(img, "Pointing", (10,100), cv2.FONT_HERSHEY_PLAIN, 3, (255,255,255), 3)
                
                if(Pencil):
                    # print(new_point.z)
                    # wr.write(new_point)
                    Writers[-1].write(new_point)
                last_time_pointer=True
            else:
                last_time_pointer=False
        # Draw(img,wr.points) 
        # wr.draw(img)
        for Writer in Writers:
            img=Writer.draw(img)         
                    
        

        cTime = time.time()
        fps = 1/(cTime-pTime)
        pTime = cTime


        

        cv2.putText(img,str(int(fps)), (10,70), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,255), 3)
   
       
        cv2.imshow("Image", img)
        cv2.waitKey(1)

      

        if not (cv2.getWindowProperty("Image",cv2.WND_PROP_VISIBLE)):
            break
    
        if cv2.waitKey(1) == 27: 
            break  # esc to quit

    except Exception as e:
        print(e)
        break
cap.release()
cv2.destroyAllWindows()
    